<img src="images/Hackathon mobilités_logo simple.jpg" alt="Logo Hackathon 2025" width="400"/>

## Snippets de code 

Ces snippets de code vont vous permettre de gagner du temps dans la prise en main des ressources à disposition et de l'écosystème data d'île-de-France Mobilités. 

In [ ]:
!pip install -r requirements.txt

## Sommaire

- [Appel d'API PRIM — Instanciation API](#appel-dapi-prim)
  - [Nomenclature pour les appels API](#nomenclature-pour-les-appels-api)
    - [Code ligne vers identifiant SIRI](#code-ligne-vers-identifiant-de-ligne-siri)
    - [Code arrêt vers identifiant d'arrêt SIRI](#code-arrêt-vers-identifiant--darrêt-siri)
  - [Appel d'API PRIM — Exemple de calcul d'itinéraire](#exemple-de-demande-de-calcul-ditinéraire)
  - [Appel d'API PRIM — Autres exemples d'appels vers des endpoints](#autres-exemples-dappels-vers-des-endpoints)
- [Onyxia — Connexion au stockage](#connexion-au-stockage)
  - [Lecture S3](#lecture-dun-fichier-depuis-lespace-de-stockage-partagé-vers-une-dataframe-pandas)
  - [Ecriture dataframe vers S3](#ecriture-dun-fichier-depuis-une-dataframe-pandas-vers-lespace-de-stockage-partagé)
  - [Ecriture fichiers vers S3](#ecriture-de-plusieurs-fichiers-au-sein-dun-dossier-vers-lespace-de-stockage-partagé)
- [Utilisation de modèles ouverts sur Hugging Face en local](#utilisation-de-modèles-ouverts-sur-hugging-face-en-local)
- [Utilisation de modèles OpenAI déployés par IDFM sur Azure](#utilisation-de-modèles-deployés-par-idfm-sur-azure)
  - [Appel d'un LLM](#appel-dun-llm)
  - [Appel d'un modèle d'embedding](#appel-dun-modèle-dembedding)

*Cliquez sur un titre (si votre visualiseur de notebook supporte les ancres) pour accéder à la section.*

### Appel d'API PRIM

Île de France Mobilités met à disposition une plateforme pour interroger ses API d'information voyageurs. Cette plateforme nécessite la création d'un token et comporte quelques subtilités dans les codifications des lignes. Des éléments sont fournis ci-dessous pour vous en faciliter l'utilisation.

<img src="images/api_prim.png" alt="Token authentification PRIM" width="900"/>


In [ ]:
import requests
import json
import pandas as pd

# Informations API
API_BASE = "https://prim.iledefrance-mobilites.fr/marketplace/"
API_KEY = "your-api-key"  # remplacer par votre clé
HEADERS = {"Accept": "application/json", "apikey": API_KEY}

#### Nomenclature pour les appels API
##### Code ligne vers identifiant de ligne SIRI

In [ ]:
# Conversion de l'id_ligne du référentiel des lignes en id compatible avec les API
# /!\ les "deux points" finaux sont indispensables

code_ligne_idfm = "C01742"  # RER A
id_ligne_idfm = "STIF:Line::{id_ligne_idfm}:"

##### Code arrêt vers identifiant  d'arrêt SIRI
_Note : les identifiant d'arrêts à prendre en compte sont en général les id de zones d'arrêt._

In [ ]:
# Conversion de ZdAId du référentiel des arrêts en id compatible avec les API
# /!\ les "deux points" finaux sont indispensables

code_zone_arret_idfm = "42135"  # Les Dix Arpents
id_arret_idfm = "STIF:monomodalStopPlace::{code_zone_arret_idfm}:"

#### Exemple de demande de calcul d'itinéraire

In [ ]:
API_URL = API_BASE + "v2/navitia/journeys"

# Coordonnées (lon, lat) — des chaînes conviennent pour le tutoriel
ORIGIN_LON = "2.33792"
ORIGIN_LAT = "48.85827"
DEST_LON = "2.3588523"
DEST_LAT = "48.9271087"

# Date/heure pour le trajet (format : YYYYMMDDTHHMMSS)
TRIP_DATETIME = "20241121T073000"

# Construire l'URL (Navitia attend lon;lat encodé en lon%3B%20lat)
FROM_PARAM = f"{ORIGIN_LON}%3B%20{ORIGIN_LAT}"
TO_PARAM = f"{DEST_LON}%3B%20{DEST_LAT}"
URL = f"{API_URL}?from={FROM_PARAM}&to={TO_PARAM}&datetime={TRIP_DATETIME}"

# Afficher l'URL pour que le lecteur voie comment elle est construite
print("Aperçu de l'URL Navitia :")
print(URL)

In [ ]:
# Exécuter la requête inline (style tutoriel simple)
try:
    resp = requests.get(URL, headers=HEADERS)
    print("Statut HTTP :", resp.status_code)

    if resp.status_code == 200:
        data = resp.json()
        # Aplatir le JSON de premier niveau pour inspection
        df = pd.json_normalize(data)
        print("Clés de premier niveau :", list(data.keys()))

        # Si des 'journeys' sont présents, inspecter le premier
        if isinstance(data, dict) and data.get("journeys"):
            print("Premier trajet (aplatit) :")
            display(pd.json_normalize(data["journeys"][0]))
        else:
            print("Aucun trajet ('journeys') retourné dans la réponse.")
    else:
        print("Réponse non-200, corps (400 premiers caractères) :")
        print(resp.text[:400])

except Exception as e:
    print("La requête a échoué :", e)

#### Autres exemples d'appels vers des endpoints

Ci-dessous un exemple avec l'endpoint general-message qui renvoie des messages info trafic.

In [ ]:
ID_LIGNE = "C01742"  # Exemple : RER A
API_URL = f"{API_BASE}general-message?LineRef=STIF%3ALine%3A%3A{ID_LIGNE}%3A"

# Affichage rapide pour vérification (utile en mode tutoriel)
print("Requête vers :", API_URL)

# --- Requête inline et traitement minimal ---
try:
    response = requests.get(API_URL, headers=HEADERS)
    print("Statut HTTP :", response.status_code)

    if response.status_code == 200:
        # Parser le JSON et l'afficher (ou l'extraire selon le besoin)
        json_data = response.json()
        print("Réponse JSON (extrait) :")
        # Affiche la structure JSON (ou un extrait) pour inspection
        print(json.dumps(json_data, indent=2, ensure_ascii=False)[:1000])
    else:
        print("Échec de la requête HTTP. Statut :", response.status_code)
        print("Réponse (début) :", response.text[:400])

except Exception as e:
    print("Une erreur s'est produite lors de l'appel HTTP :", str(e))

### Onyxia
#### Connexion au stockage

<img src="images/acces_s3_onyxia.png" alt="Accès S3 Onyxia" width="900"/>


In [ ]:
import io
import boto3
import pandas as pd

# Remplacer par vos propres informations d'identification et paramètres
ACCESS_KEY = "your-access-key"
SECRET_KEY = "your-secret-key"
SESSION_TOKEN = "your-session-token"
REGION = "fr-central"
ENDPOINT_URL = "minio.data-platform-self-service.net"

BUCKET = "dlb-hackathon"

s3 = boto3.client(
    "s3",
    endpoint_url="https://" + ENDPOINT_URL,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    aws_session_token=SESSION_TOKEN,
    region_name=REGION,
)

response = s3.list_objects_v2(Bucket="dlb-hackathon")
if "Contents" in response:
    for obj in response["Contents"]:
        if obj["Key"].startswith("datasets-diffusion/2025"):
            print(obj["Key"])

#### Lecture d'un fichier depuis l'espace de stockage partagé vers une dataframe pandas

_RAPPEL : pour plus d'informations sur un jeu de données spécifique (structure, source, exemple de cas d'usage en accord avec les défis, etc.), prenez le temps de lire le README.md présent dans le dossier associé._

In [ ]:
FILE_KEY_S3 = "/datasets-diffusion/2024/ascenseurs_historique_etat/RELEVES_ETATS_ASCENSEURS_SNCF_RATP_2021-2024.csv"

response = s3.get_object(Bucket=BUCKET, Key=FILE_KEY_S3)
status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

if status == 200:
    df = pd.read_csv(response.get("Body"))
    print("Aperçu des données importées :")
    print(df.head())
else:
    print(f"Erreur lors de l'importation des données. Statut HTTP: {status}")

#### Ecriture d'un fichier depuis une dataframe pandas vers l'espace de stockage partagé

In [ ]:
FILE_KEY_S3 = "tests_io_onyxia/books.csv"
books_df = pd.DataFrame(
    data={"Title": ["Book I", "Book II", "Book III"], "Price": [56.6, 59.87, 74.54]},
    columns=["Title", "Price"],
)

with io.StringIO() as csv_buffer:
    books_df.to_csv(csv_buffer, index=False)
    response = s3.put_object(Bucket=BUCKET, Key=FILE_KEY_S3, Body=csv_buffer.getvalue())

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")
    if status == 200:
        print("Fichier écrit avec succès dans l'espace de stockage partagé.")
    else:
        print(f"Erreur lors de l'écriture du fichier. Statut HTTP: {status}")

#### Ecriture de plusieurs fichiers au sein d'un dossier vers l'espace de stockage partagé

In [ ]:
import os

LOCAL_FOLDER = "C:/Users/.../exploration_dept_data/0_donnees_2025"
PREFIX = "datasets-diffusion/2025"

paginator = s3.get_paginator("list_objects_v2")

for root, dirs, files in os.walk(LOCAL_FOLDER):
    for filename in files:
        local_path = os.path.join(root, filename)
        relative_path = os.path.relpath(local_path, LOCAL_FOLDER)
        s3_key = f"{PREFIX}/{relative_path.replace(os.sep, '/')}"
        
        print(f"Uploading {local_path} → s3://{BUCKET}/{s3_key}")
        s3.upload_file(local_path, BUCKET, s3_key)

### Utilisation de modèles ouverts sur Hugging Face en local

La plateforme permet d'explorer l'utilisation de petits modèles. Ces modèles ne sont exécutés qu'avec du CPU car la plateforme ne dispose pas de GPU. Il est recommandé d'explorer des modèles de petite taille, avec moins de 500 millions ou un milliard de paramètres, car au-delà, le temps d'exécution serait trop long et cela perturberait le fonctionnement général de la plateforme.

In [ ]:
from llama_cpp import Llama

# Cette instruction minimale permet de sélectionner le modèle et son niveau de compression, le téléchargement le paramétrage et la mise en mémoire (CPU), sont automatiques :
llm = Llama.from_pretrained(
    repo_id="bartowski/gemma-2-2b-it-GGUF",  # Plus gros dépôt de LLM libres de Hugging face : https://huggingface.co/bartowski
    filename="gemma-2-2b-it-Q6_K_L.gguf",  # Attention a prendre des modèles de taille <= 6Go pour des performances raisonnables, pas de garantie de fonctionnement au delà
    n_ctx=4096,
    verbose=False,
)

# Une fois le LLM chargé (environ une minute par Go), il peut être appelé, via une fonction *completion, ici pour un chatbot :
llm_response = llm.create_chat_completion(
    messages=[
        {
            "role": "user",
            "content": "###SYSTEM : You are the Capital assistant. I give you a country, you give me the capital and try to retrieve the country iso code. ### FORMAT : Capital=<name of the Capital> CODE=<ISO CODE 2 CHARS>",
        },
        {
            "role": "assistant",
            "content": "OK, I understand. If you say France, i say Capital=Paris CODE=FR",
        },
        {"role": "user", "content": "Germany"},
    ]
)

# La réponse du LLM est pleine de métadonnées, mais le message texte peut être extrait de cette manière simple :
print(llm_response["choices"][0]["message"]["content"])

### Utilisation de modèles deployés par IDFM sur Azure
 
Documentation utile :
- https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/responses?tabs=python-key
- https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/chatgpt
 
Les urls des endpoints de vos équipes respectives ainsi que les tokens d'authentification associés vous seront communiqués via Slack.
 
Ci-après la liste des modèles et versions disponibles :
* o4-mini / 2025-04-16
* gpt-4o-mini / 2024-07-18
* Llama-4-Maverick-17B-128E-Instruct-FP8 / 1
* text-embedding-3-large / 1
 
Les **quotas de requêtes** sont les suivants :
- LLM : 50000 tokens/minute
- Embeddings : 100000 tokens/minute

#### Appel d'un LLM

In [ ]:
from openai import OpenAI

endpoint = "votre_endpoint_d_equipe"+"openai/v1/"
api_key = "votre_cle_d_equipe"

model_name = "gpt-4o-mini" #"Llama-4-Maverick-17B-128E-Instruct-FP8" #"gpt-4o-mini" #"o4-mini" #"Llama-4-Maverick-17B-128E-Instruct-FP8"


client = OpenAI(
    base_url=f"{endpoint}",
    api_key=api_key
)
completion = client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": "What were the capitals of France?",
        }
    ],
    max_completion_tokens=13107,
    temperature=1,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
)
print(completion.choices[0].message.content)

#### Appel d'un modèle d'embedding

In [ ]:
import os
 
endpoint = "votre_endpoint_d_equipe"+"openai/v1/"
api_key = "votre_cle_d_equipe"

model_name =model_name = "text-embedding-3-large"
 
client = OpenAI(
    base_url=endpoint,
    api_key=api_key
)
response = client.embeddings.create(
    input=["first phrase","second phrase","third phrase"],
    model=model_name
)
for item in response.data:
    length = len(item.embedding)
    print(
        f"data[{item.index}]: length={length}, "
        f"[{item.embedding[0]}, {item.embedding[1]}, "
        f"..., {item.embedding[length-2]}, {item.embedding[length-1]}]"
    )
print(response.usage)